In [32]:
# Data handling and processing
import pandas as pd
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Sklearn imports
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.metrics import accuracy_score

# Model imports
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# File mgmt
import os

# misc
import time

os.chdir('Data')
!ls

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'Data'

In [7]:
path = 'Transactions_Village114/ds3170_tx_All_Data_5208_2019_1211_082948.txt'
df = pd.read_csv(path, sep="\t")
cols = ['Anon Student Id', 'Session Id', 'Duration (sec)', 'Level (Tutor Name)', 'Level (Tutor)', 'Problem Name', 'Problem View','Step Name', 'Attempt At Step','Is Last Attempt','Outcome', 'Input','CF (File)','CF (Matrix)','CF (Matrix Level)', 'CF (Matrix Order)', 'CF (Total Activity Problems)']
df = df[cols]
bubble_pop_df = df[df['Level (Tutor Name)'] == 'bubble_pop']
bpop_math_df = bubble_pop_df[bubble_pop_df['CF (Matrix)'] == 'math'] #6172

# X and y -> inputs and outputs for the classification model
cols = ['Duration (sec)', 'Level (Tutor)', 'Attempt At Step', 'CF (Matrix Level)', 'CF (Matrix Order)', 'Outcome']
X = bpop_math_df[cols]
y = X[['Outcome']]
X = X.drop(['Outcome'], axis=1)

C:\Users\jithe\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,7,37,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Helpful functions

In [145]:
# Clean duration (sec) columns: Convert all values to float, if not able to convert set value in that row as duration_mean
def clean_duration_col_with_mean(X):
    strings = 0
    ints = 0
    floats = 0
    other = 0
    cnt = 0
    sums = 0
    posn = []
    X_duration_arr = X['Duration (sec)'].array

    for i in range(len(X_duration_arr)):
        if isinstance(X_duration_arr[i], int):
            ints += 1
        elif isinstance(X_duration_arr[i], str):
            strings += 1
            try:
                X_duration_arr[i] = float(X_duration_arr[i])
            except ValueError:
#                 print("Cant convert to a float", X_duration_arr[i], "Position: ",i)
                posn.append(i)
        elif isinstance(X_duration_arr[i], float):
            floats += 1
    
        if isinstance(X_duration_arr[i], float):
            sums += X_duration_arr[i]
            cnt += 1

    duration_mean = sums/cnt
    for i in range(len(posn)):
        X_duration_arr[posn[i]] = duration_mean

    for i in range(len(X_duration_arr)):
        col = bpop_math_df.columns.get_loc('Duration (sec)')
        val = X_duration_arr[i]
        X.iloc[i, col] = val
    
    return X

# Ordinal encodes a column
def ordinal_encode_col(X, colname):
    enc = OrdinalEncoder()
    level_tutor = X[colname].tolist()
    tutor_level_2dlist = []
    for i in range(len(level_tutor)):
        tutor_level_2dlist.append([level_tutor[i]])

    enc.fit(tutor_level_2dlist)

    col = X.columns.get_loc(colname)
    num_entries = X.count()[colname]
    for i in range(num_entries):
        val = X.iloc[i, col]
        val = enc.transform([[val]])[0][0]
        X.iloc[i, col] = val
    
    return X

def rfe(X_train, y_train, apply_rfe):
    if apply_rfe == False:
        return
    data_final_vars=X_train.columns.values.tolist()
    y_=['y']
    X=[i for i in data_final_vars if i not in y_]
    logreg = LogisticRegression()
    rfe = RFE(logreg, 20)
    rfe = rfe.fit(X_train, y_train)
#     print("RFE Support: ",rfe.support_)
#     print("RFE Ranking: ", rfe.ranking_)

# incorrect -> 0, correct -> 1
# set_zero = "INCORRECT" means incorrect will be set to 0 and other value as 1
def encode_outputs(y, colname, set_zero):
    col = y.columns.get_loc(colname)
    for i in range(y.count()[colname]): 
        val = y.iloc[i, col]
        if val == set_zero:
            val = 0
        else: 
            val = 1
        y.iloc[i, col] = val
    
    return y

def encode_categorical_cols(X, encoder, col):

#   Ordinally encode categorical columns
    if encoder == "ordinal":
        for i in range(len(col)):
            X = ordinal_encode_col(X, col[i])
    
    return X

def normalize_data(X, norm):
    if norm == 'l1':
        X = sklearn.preprocessing.normalize(X, norm='l1', axis=1, copy=True, return_norm=False)
        X = pd.DataFrame(data=X, columns=cols[:len(cols)-1])
    
    elif norm == 'l2':
        X = sklearn.preprocessing.normalize(X, norm='l2', axis=1, copy=True, return_norm=False)
        X = pd.DataFrame(data=X, columns=cols[:len(cols)-1])
    
    return X


def run_logistic_regression(X, y, test_split, apply_rfe):
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split)
    rfe(X_train, y_train, apply_rfe)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    performance = 100 * logreg.score(X_test, y_test)
#     print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(performance))
    return performance
    
def run_k_neighbors_classifier(X, y, test_split, apply_rfe, neighbors):
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split)
    rfe(X_train, y_train, apply_rfe)       
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(X_train, y_train)
    performance = 100 * neigh.score(X_test, y_test)
#     print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(performance))
    return performance

def run_svm(X, y, test_split, apply_rfe, svc_kernel):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split)
    rfe(X_train, y_train, apply_rfe)   
    svm_model = svm.SVC(kernel=svc_kernel)
    print("CREATE MODEL")
    svm_model.fit(X_train, y_train)
    print("FIT")
    #Predict the response for test dataset
    y_pred = svm_model.predict(X_test)
    perf = 100 * metrics.accuracy_score(y_test, y_pred)
    print("Accuracy:",perf)
    return perf

def run_nn(X, y, test_split, lr, epochs):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Make everything as float type

    X_train['Duration (sec)'] = X_train['Duration (sec)'].astype(float)
    X_train['Level (Tutor)'] = X_train['Level (Tutor)'].astype(float)
    X_train['CF (Matrix Level)'] = X_train['CF (Matrix Level)'].astype(float)
    X_train['CF (Matrix Order)'] = X_train['CF (Matrix Order)'].astype(float)

    X_test['Duration (sec)'] = X_test['Duration (sec)'].astype(float)
    X_test['Level (Tutor)'] = X_test['Level (Tutor)'].astype(float)
    X_test['CF (Matrix Level)'] = X_test['CF (Matrix Level)'].astype(float)
    X_test['CF (Matrix Order)'] = X_test['CF (Matrix Order)'].astype(float)

    # Convert to torch tensors

    X_train_tensor = torch.tensor(X_train.values).float()
    X_test_tensor = torch.tensor(X_test.values).float()
    y_train_tensor = torch.tensor(y_train).float()
    y_test_tensor = torch.tensor(y_test).float()
    
    
    # Model class
    class StudentNet(nn.Module):
    
        def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
            super(StudentNet, self).__init__()
            self.fc1 = nn.Linear(5, hidden1_size)
            self.fc2 = nn.Linear(hidden1_size, hidden2_size)
            self.fc3 = nn.Linear(hidden1_size, num_classes)
    
        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x
    
        def predict(self,x):
            pred = F.softmax(self.forward(x))
            ans = []
            #Pick the class with maximum proba
            for t in pred:
                if t[0]>t[1]:
                    ans.append(0)
                else:
                    ans.append(1)
            return torch.tensor(ans)
    
    model = StudentNet(5, 20, 20, 2)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss()
    
    for i in range(epochs):
    
        #Predict the output for Given input
        y_pred = model.forward(X_train_tensor)
        loss = loss_function(y_pred, y_train_tensor.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i%(epochs/10) == 0:
            print("Loss: ", loss.item())

    perf = 100 * accuracy_score(model.predict(X_test_tensor),y_test_tensor)
    print("ACCURACY: ", perf, "%")
    return perf
    

def run_algo(algo, X, y, clean_method, encoder, norm, test_split, apply_rfe, categorical_cols, neighbors, svc_kernel, lr, epochs):
    
    if clean_method == "mean":
        X = clean_duration_col_with_mean(X)
    
    X = encode_categorical_cols(X, encoder, categorical_cols)
    X = normalize_data(X, norm)
    y = encode_outputs(y, 'Outcome', 'INCORRECT').values.ravel()
    perf = 0
    if algo == "logistic_regression":
        perf = run_logistic_regression(X, y, test_split, apply_rfe)
    
    elif algo == "k_neighbors_classifier":
        perf = run_k_neighbors_classifier(X, y, test_split, apply_rfe, neighbors)
    
    elif algo == "svm":
        perf = run_svm(X, y, test_split, apply_rfe, svc_kernel)
        
    elif algo == "nn":
        perf = run_nn(X, y, test_split, lr, epochs)
      
    return perf
    

In [148]:
# mean median or mode (#FIXME: only supports mean for now)
clean_method = "mean" 

# ordinal, onehot etc.. (#FIXME: only supports ordinal encoding for now)
encoder = "ordinal" 

# cols that are categories rather than numbers and have to be encoded
categorical_cols = ['Level (Tutor)']

# none, l1, l2 etc..
norm = "none" 

# set True to apply Recursive Feature Elimination
apply_rfe = True

test_split = 0.15
num_runs = 10
lr = 0.08
epochs = 5000

# k neighbors classifier
neighbors = 5

# linear, poly, rbf, sigmoid
svc_kernel = 'linear'

# Supports logistic_regression, k_neighbors_classifier, svm, nn
algo = "nn"

# used to store performance of each run in num_runs
total_perfs = []

for i in range(num_runs):
    X = bpop_math_df[cols]
    X = X.sample(frac=1).reset_index(drop=True)
    y = X[['Outcome']]
    X = X.drop(['Outcome'], axis=1)
    perf = run_algo(algo, X, y, clean_method, encoder, norm, test_split, apply_rfe, categorical_cols, neighbors, svc_kernel, lr, epochs)
    total_perfs.append(perf)

avg_perf = sum(total_perfs)/num_runs
print("Avg. performance over ", num_runs, "runs: ", avg_perf)
print(avg_perf, "%(", min(total_perfs), "-", max(total_perfs) ,")")

KeyboardInterrupt: 